In [1]:
import numpy as np
import h5py
from scipy.sparse import coo_matrix, csr_matrix
import pandas as pd
import scipy.io
import scipy.sparse as sparse
import anndata
import deeplift
import shap
from deeplift.conversion import kerasapi_conversion as kc
from scbasset.utils import *
from scbasset.basenji_utils import *
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import custom_object_scope
import json

/tmp/ipykernel_66561/3711617372.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/pascal/Norway/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-19 15:53:27.252890: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-19 15:53:27.293955: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factor

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
input_model_path = '../../../results_scbasset/promotor/model/l2_regularization/0_5/test_new_model/model_hope1.h5'
json_model_path = '../../../results_scbasset/promotor/model/l2_regularization/0_5/test_new_model/hope.json'
autre_model= '../../../results_scbasset/promotor/model/l2_regularization/0_5/test_new_model/model_without_layers.h5'
weight_path ='../../../results_scbasset/promotor/model/l2_regularization/0_5/test_new_model/best_model.h5'

### Change functional to model


In [ ]:
import h5py
import json
  # Chemin vers votre fichier modèle .h5
new_class_name = 'Model'  # Le nouveau nom de classe que vous voulez utiliser

# Étape 1 : Ouvrir le fichier .h5 en mode lecture/écriture
with h5py.File(input_model_path, 'r+') as h5_file:
    # Étape 2 : Lire la configuration du modèle sous forme de chaîne JSON
    model_config = json.loads(h5_file.attrs['model_config'])
    
    # Étape 3 : Charger la chaîne JSON dans un objet Python (dictionnaire)
    # Étape 4 : Modifier le nom de la classe dans le dictionnaire
    model_config['class_name'] = new_class_name
    
    # Étape 5 : Convertir le dictionnaire modifié en chaîne JSON
    modified_model_config_str = json.dumps(model_config)
    
    
    # Étape 6 : Mettre à jour les attributs du modèle avec la nouvelle chaîne JSON
    h5_file.attrs['model_config'] = modified_model_config_str.encode('utf-8')
h5_file.close()



## Create Model

In [4]:
model = tf.keras.models.Sequential()
#0
model.add(tf.keras.Input(shape=(1344, 4), name='sequence'))
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Conv1D(filters=288, kernel_size=17,strides=1, padding= 'same', input_shape=(1344, 4), name='conv1d', use_bias=False))
model.add(tf.keras.layers.BatchNormalization(axis=2,
    momentum=0.9,
    epsilon=0.001,
    center=True,
    scale=True,
    beta_initializer="zeros",
    gamma_initializer="ones",
    moving_mean_initializer="zeros",
    moving_variance_initializer="ones",
    beta_regularizer=None,
    gamma_regularizer=None,
    beta_constraint=None,
    gamma_constraint=None,
    synchronized=False,
    name= 'batch_normalization'
))
model.add(tf.keras.layers.MaxPooling1D(
    pool_size=3, 
    strides=3, 
    padding="same",
    data_format="channels_last",
    name='max_pooling1d',
))
#1
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Conv1D(filters=288, kernel_size=5,
                              strides=1, use_bias=False, padding='same', name="conv1d_1"))
model.add(tf.keras.layers.BatchNormalization(axis=2,
    momentum=0.9,
    epsilon=0.001,
    center=True,
    scale=True,
    beta_initializer="zeros",
    gamma_initializer="ones",
    moving_mean_initializer="zeros",
    moving_variance_initializer="ones",
    beta_regularizer=None,
    gamma_regularizer=None,
    beta_constraint=None,
    gamma_constraint=None,
    synchronized=False,
    name= 'batch_normalization_1'
))
model.add(tf.keras.layers.MaxPooling1D(
    pool_size=2, 
    strides=2, 
    padding="same",
    data_format="channels_last",
    name='max_pooling1d_1',
))
#2
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Conv1D(filters=323, kernel_size=5,
                              strides=1, use_bias=False, padding='same', name="conv1d_2"))
model.add(tf.keras.layers.BatchNormalization(axis=2,
    momentum=0.9,
    epsilon=0.001,
    center=True,
    scale=True,
    beta_initializer="zeros",
    gamma_initializer="ones",
    moving_mean_initializer="zeros",
    moving_variance_initializer="ones",
    beta_regularizer=None,
    gamma_regularizer=None,
    beta_constraint=None,
    gamma_constraint=None,
    synchronized=False,
    name= 'batch_normalization_2'
))
model.add(tf.keras.layers.MaxPooling1D(
    pool_size=2, 
    strides=2, 
    padding="same",
    data_format="channels_last",
    name='max_pooling1d_2',
))
#3
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Conv1D(filters=363, kernel_size=5,
                              strides=1, use_bias=False,  padding='same',name="conv1d_3"))
model.add(tf.keras.layers.BatchNormalization(axis=2,
    momentum=0.9,
    epsilon=0.001,
    center=True,
    scale=True,
    beta_initializer="zeros",
    gamma_initializer="ones",
    moving_mean_initializer="zeros",
    moving_variance_initializer="ones",
    beta_regularizer=None,
    gamma_regularizer=None,
    beta_constraint=None,
    gamma_constraint=None,
    synchronized=False,
    name= 'batch_normalization_3'
))
model.add(tf.keras.layers.MaxPooling1D(
    pool_size=2, 
    strides=2, 
    padding="same",
    data_format="channels_last",
    name='max_pooling1d_3',
))
#4
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Conv1D(filters=407, kernel_size=5,
                              strides=1, use_bias=False, padding='same', name="conv1d_4"))
model.add(tf.keras.layers.BatchNormalization(axis=2,
    momentum=0.9,
    epsilon=0.001,
    center=True,
    scale=True,
    beta_initializer="zeros",
    gamma_initializer="ones",
    moving_mean_initializer="zeros",
    moving_variance_initializer="ones",
    beta_regularizer=None,
    gamma_regularizer=None,
    beta_constraint=None,
    gamma_constraint=None,
    synchronized=False,
    name= 'batch_normalization_4'
))
model.add(tf.keras.layers.MaxPooling1D(
    pool_size=2, 
    strides=2, 
    padding="same",
    data_format="channels_last",
    name='max_pooling1d_4',
))
#5
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Conv1D(filters=456, kernel_size=5,
                              strides=1, use_bias=False, padding='same', name="conv1d_5"))
model.add(tf.keras.layers.BatchNormalization(axis=2,
    momentum=0.9,
    epsilon=0.001,
    center=True,
    scale=True,
    beta_initializer="zeros",
    gamma_initializer="ones",
    moving_mean_initializer="zeros",
    moving_variance_initializer="ones",
    beta_regularizer=None,
    gamma_regularizer=None,
    beta_constraint=None,
    gamma_constraint=None,
    synchronized=False,
    name= 'batch_normalization_5'
))
model.add(tf.keras.layers.MaxPooling1D(
    pool_size=2, 
    strides=2, 
    padding="same",
    data_format="channels_last",
    name='max_pooling1d_5',
))
#6
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Conv1D(filters=512, kernel_size=5,
                              strides=1, use_bias=False, padding='same', name="conv1d_6"))
model.add(tf.keras.layers.BatchNormalization(axis=2,
    momentum=0.9,
    epsilon=0.001,
    center=True,
    scale=True,
    beta_initializer="zeros",
    gamma_initializer="ones",
    moving_mean_initializer="zeros",
    moving_variance_initializer="ones",
    beta_regularizer=None,
    gamma_regularizer=None,
    beta_constraint=None,
    gamma_constraint=None,
    synchronized=False,
    name= 'batch_normalization_6'
))
model.add(tf.keras.layers.MaxPooling1D(
    pool_size=2,
    strides=2, 
    padding="same",
    data_format="channels_last",
    name='max_pooling1d_6',
))
#7
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Conv1D(filters=256, kernel_size=1,
                              strides=1, use_bias=False, padding='same',  name="conv1d_7"))
model.add(tf.keras.layers.BatchNormalization(axis=2,
    momentum=0.9,
    epsilon=0.001,
    center=True,
    scale=True,
    beta_initializer="zeros",
    gamma_initializer="ones",
    moving_mean_initializer="zeros",
    moving_variance_initializer="ones",
    beta_regularizer=None,
    gamma_regularizer=None,
    beta_constraint=None,
    gamma_constraint=None,
    synchronized=False,
    name= 'batch_normalization_7'
))
#8
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Flatten(name='flatten'))
model.add(tf.keras.layers.Dense(units=32,use_bias=False, activation= "linear", name='dense'))
model.add(tf.keras.layers.BatchNormalization(axis=1,
    momentum=0.9,
    epsilon=0.001,
    center=True,
    scale=True,
    beta_initializer="zeros",
    gamma_initializer="ones",
    moving_mean_initializer="zeros",
    moving_variance_initializer="ones",
    beta_regularizer=None,
    gamma_regularizer=None,
    beta_constraint=None,
    gamma_constraint=None,
    synchronized=False,
    name= 'batch_normalization_8'
))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Activation("relu"))
model.add(tf.keras.layers.Dense(units = 98, activation='sigmoid', use_bias=True, name='dense_1'))
model.add(tf.keras.layers.Flatten(data_format='channels_last', name='flatten_1'))

model.build((None, 1344, 4))


Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
model.summary()

## AD weight

In [5]:
model.load_weights(weight_path, by_name=True)
model.save(input_model_path)



2024-04-19 15:13:33.565339: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-04-19 15:13:33.586878: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-04-19 15:13:33.625535: W tensorflow/c/c_api.cc:305] Operation '{name:'batch_normalization_8/beta/Assign' id:897 op device:{requested: '', assigned: ''} def:{{{node batch_normalization_8/beta/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](batch_normalization_8/beta, batch_normalization_8/beta/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
/home/pascal/Norway/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are 

## Create json of model

In [5]:
f = h5py.File(input_model_path)
model_config = json.loads(f.attrs['model_config'])

with open(json_model_path, "w") as json_file:
    json.dump(model_config, json_file, indent=4)
f.close()



In [45]:
model_weights = h5py.File(input_model_path)
model_weights.keys()
model_weights=model_weights['model_weights']
model_weights
tr_data = h5py.File(input_model_path).attrs["model_config"]
model_class_and_config=json.loads(open(json_model_path).read())
model_class_name = model_class_and_config["class_name"] 
model_class_name
model_config = model_class_and_config["config"]
model_config
layer_configs = model_config
layer_configs['layers'].attrs["name"]
# layer_name= 'layers'
# model_weights[layer_name]
# print("layer weight names:", [layer_name+"/"+x
#                               for layer_name in f['model_weights'].keys()
#                               for x in f['model_weights'][layer_name].attrs['weight_names']],"\n")
# for layer_config in layer_configs:
#     print(layer_config)
    #layer_name = layer_config["config"]["name"]
# # model_weights[layer_name].attrs["weight_names"]


AttributeError: 'list' object has no attribute 'attrs'

## DeepLIFT

In [4]:
deeplift_model = kc.convert_model_from_saved_files(
        input_model_path,
        json_model_path,
        nonlinear_mxts_mode=deeplift.layers.NonlinearMxtsMode.DeepLIFT_GenomicsDefault) 


TypeError: can only concatenate str (not "KeysViewHDF5") to str

## View order of the layers

In [36]:
str_data = h5py.File(input_model_path).attrs["model_config"]
model_class_and_config = json.loads(str_data)
model_config = model_class_and_config["config"]
layer_configs = model_config["layers"]
for layer_config in layer_configs:
    layer_name = layer_config["config"]["name"]
    print(layer_name.lower())
#model_config



sequence
activation
conv1d
batch_normalization
max_pooling1d
activation_1
conv1d_1
batch_normalization_1
max_pooling1d_1
activation_2
conv1d_2
batch_normalization_2
max_pooling1d_2
activation_3
conv1d_3
batch_normalization_3
max_pooling1d_3
activation_4
conv1d_4
batch_normalization_4
max_pooling1d_4
activation_5
conv1d_5
batch_normalization_5
max_pooling1d_5
activation_6
conv1d_6
batch_normalization_6
max_pooling1d_6
activation_7
conv1d_7
batch_normalization_7
activation_8
flatten
dense
batch_normalization_8
dropout
activation_9
dense_1
flatten_1


## View all json 

In [5]:
f = h5py.File(input_model_path)
print("keras version", f.attrs['keras_version'])
print("keras json", json.dumps(json.loads(f.attrs['model_config']),
                               indent=4),"\n")
print("layer weight names:", [layer_name+"/"+x
                              for layer_name in f['model_weights'].keys()
                              for x in f['model_weights'][layer_name].attrs['weight_names']],"\n")
f.close()

keras version 2.15.0
keras json {
    "class_name": "Sequential",
    "config": {
        "name": "sequential",
        "layers": [
            {
                "class_name": "InputLayer",
                "config": {
                    "batch_input_shape": [
                        null,
                        1344,
                        4
                    ],
                    "dtype": "float32",
                    "sparse": false,
                    "ragged": false,
                    "name": "sequence"
                }
            },
            {
                "class_name": "Activation",
                "config": {
                    "name": "activation",
                    "trainable": true,
                    "dtype": "float32",
                    "activation": "relu"
                }
            },
            {
                "class_name": "Conv1D",
                "config": {
                    "name": "conv1d",
                    "trainable": true,
    

## View weight of model

In [8]:
import h5py

filename = weight_path

with h5py.File(filename, 'r') as f:
    def print_name_and_shape(name, obj):
        if isinstance(obj, h5py.Dataset):  
            print("Weight name :", name, "of shape", obj.shape)
        elif isinstance(obj, h5py.Group):  
            print("Groupe:", name)
            
    f.visititems(print_name_and_shape)
f.close()


Groupe: activation
Groupe: activation_1
Groupe: activation_2
Groupe: activation_3
Groupe: activation_4
Groupe: activation_5
Groupe: activation_6
Groupe: activation_7
Groupe: activation_8
Groupe: activation_9
Groupe: batch_normalization
Groupe: batch_normalization/batch_normalization
Weight name : batch_normalization/batch_normalization/beta:0 of shape (288,)
Weight name : batch_normalization/batch_normalization/gamma:0 of shape (288,)
Weight name : batch_normalization/batch_normalization/moving_mean:0 of shape (288,)
Weight name : batch_normalization/batch_normalization/moving_variance:0 of shape (288,)
Groupe: batch_normalization_1
Groupe: batch_normalization_1/batch_normalization_1
Weight name : batch_normalization_1/batch_normalization_1/beta:0 of shape (288,)
Weight name : batch_normalization_1/batch_normalization_1/gamma:0 of shape (288,)
Weight name : batch_normalization_1/batch_normalization_1/moving_mean:0 of shape (288,)
Weight name : batch_normalization_1/batch_normalization_

## Add name sequence in weight file

In [4]:
filename = input_model_path

# Ouvrir le fichier en mode lecture/écriture sans écraser le contenu existant
with h5py.File(filename, 'r+') as f:
    # Créer le groupe 'sequence' s'il n'existe pas déjà
    if 'sequence' not in f:
        f.create_group('sequence')
    
    # Obtenez une liste de tous les éléments (groupes et datasets) dans le fichier
    all_items = list(f.items())
    
    # Déplacez chaque élément dans le groupe 'sequence'
    for name, obj in all_items:
        # Ignorez le groupe 'sequence' lui-même
        if name != 'sequence':
            # Déplacez l'élément dans le groupe 'sequence'
            f.move(name, 'sequence/' + name)

    # Fonction pour afficher le nom et la forme des datasets et groupes
    def print_name_and_shape(name, obj):
        if isinstance(obj, h5py.Dataset):  
            print("Nom du poids :", name, "de forme", obj.shape)
        elif isinstance(obj, h5py.Group):  
            print("Groupe :", name)
            
    # Parcourir et afficher la structure du fichier après l'ajout du groupe
    f.visititems(print_name_and_shape)
f.close()

Groupe : sequence
Groupe : sequence/model_weights
Groupe : sequence/model_weights/activation
Groupe : sequence/model_weights/activation_1
Groupe : sequence/model_weights/activation_2
Groupe : sequence/model_weights/activation_3
Groupe : sequence/model_weights/activation_4
Groupe : sequence/model_weights/activation_5
Groupe : sequence/model_weights/activation_6
Groupe : sequence/model_weights/activation_7
Groupe : sequence/model_weights/activation_8
Groupe : sequence/model_weights/activation_9
Groupe : sequence/model_weights/batch_normalization
Groupe : sequence/model_weights/batch_normalization/batch_normalization
Nom du poids : sequence/model_weights/batch_normalization/batch_normalization/beta:0 de forme (288,)
Nom du poids : sequence/model_weights/batch_normalization/batch_normalization/gamma:0 de forme (288,)
Nom du poids : sequence/model_weights/batch_normalization/batch_normalization/moving_mean:0 de forme (288,)
Nom du poids : sequence/model_weights/batch_normalization/batch_nor

### Charger modèle

In [ ]:
with custom_object_scope({
    'StochasticReverseComplement': StochasticReverseComplement,
    'StochasticShift': StochasticShift,
    'GELU': GELU,
    'SwitchReverse': SwitchReverse,
}):
    model = load_model(input_model_path)

In [ ]:
print(tf.__version__)

In [ ]:
file_path = '../../../results_scbasset/promotor_final/processed/filtered/val_seqs.h5'

# Ouvrir le fichier HDF5 en mode lecture
with h5py.File(file_path, 'r') as f:
    # Supposons que vos données sont stockées sous le dataset nommé 'X_test'
    # Vous devez connaître le nom exact du dataset dans votre fichier
    x_test = f['X'][:]
    print("Shape of X_test:", x_test.shape)

In [ ]:
def one_hot_encode(sequences):
    # Créer un tableau numpy pour contenir l'encodage one-hot
    one_hot = np.zeros((sequences.shape[0], sequences.shape[1], 4), dtype=np.float32)

    # Encoder chaque base par one-hot
    for i in range(4):
        one_hot[:, :, i] = (sequences == i)
    
    return one_hot

# Supposons que x_test est votre matrice (1171, 1344) d'entiers
x_test_one_hot = one_hot_encode(x_test)
print("Shape of X_test after one-hot encoding:", x_test_one_hot.shape)

In [ ]:
print(x_test_one_hot)
print(x_test_one_hot.shape)

In [ ]:
# Initialisation de l'Explainer SHAP
explainer = shap.DeepExplainer(model, x_test_one_hot[:100])

# Calcul des valeurs SHAP
shap_values = explainer.shap_values(x_test_one_hot[11:20])

# Visualisation des résultats
shap.summary_plot(shap_values, x_test_one_hot[11:20])

In [ ]:
print(len(x_test_one_hot[11:20]))

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, MaxPooling1D, Activation, Flatten, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import HeNormal, Zeros, Ones
from tensorflow.keras.regularizers import L1L2

# Initializers
he_normal = HeNormal()
zeros = Zeros()
ones = Ones()

# Input layer
input_layer = Input(shape=(1344, 4), name='sequence')

# First block
x = Activation('relu', name='activation')(input_layer)
x = Conv1D(288, 17, strides=1, padding='same', use_bias=False, kernel_initializer=he_normal, name='conv1d')(x)
x = BatchNormalization(axis=2, momentum=0.9, epsilon=0.001, center=True, scale=True, name='batch_normalization')(x)
x = MaxPooling1D(3, strides=3, padding='same', name='max_pooling1d')(x)

# Subsequent blocks
activation_layers = ['activation_1', 'activation_2', 'activation_3', 'activation_4', 'activation_5', 'activation_6', 'activation_7', 'activation_8']
conv_layers = ['conv1d_1', 'conv1d_2', 'conv1d_3', 'conv1d_4', 'conv1d_5', 'conv1d_6', 'conv1d_7']
filter_sizes = [288, 323, 363, 407, 456, 512, 256]
kernel_sizes = [5, 5, 5, 5, 5, 5, 1]
pool_sizes = [2, 2, 2, 2, 2, 2, 0]  # Last layer has no pooling
batch_norm_names = ['batch_normalization_1', 'batch_normalization_2', 'batch_normalization_3', 'batch_normalization_4', 'batch_normalization_5', 'batch_normalization_6', 'batch_normalization_7']

for i in range(7):
    x = Activation('relu', name=activation_layers[i])(x)
    x = Conv1D(filters=filter_sizes[i], kernel_size=kernel_sizes[i], strides=1, padding='same', use_bias=False, kernel_initializer=he_normal, name=conv_layers[i])(x)
    x = BatchNormalization(axis=2, momentum=0.9, epsilon=0.001, center=True, scale=True, name=batch_norm_names[i])(x)
    if pool_sizes[i] > 0:
        x = MaxPooling1D(pool_sizes[i], strides=2, padding='same', name='max_pooling1d_' + str(i + 1))(x)

# Flatten and dense layers before final output
x = Flatten(name='flatten')(x)
x = Dense(32, use_bias=False, activation="linear", kernel_initializer=he_normal, name='dense')(x)
x = BatchNormalization(axis=1, momentum=0.9, epsilon=0.001, center=True, scale=True, name='batch_normalization_8')(x)
x = Dropout(0.2, name='dropout')(x)
x = Activation('relu', name='activation_9')(x)
x = Dense(98, activation='sigmoid', use_bias=True, kernel_initializer=he_normal, name='dense_1')(x)

# Final model
model = Model(inputs=input_layer, outputs=x)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

weight_path ='../../../results_scbasset/promotor/model/l2_regularization/0_5/test_new_model/best_model.h5'
model.load_weights(weight_path, by_name=True)


model.summary()


model.save(input_model_path)

